In [1]:
from perceval.backends.core.github import GitHub
from datetime import date
import datetime
from datetime import datetime as dt
import dateutil.tz
from dateutil.relativedelta import relativedelta
import matplotlib.pyplot as plt
import pandas as pd
import os
import sqlalchemy as s
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import scoped_session, sessionmaker
import augur
from time import gmtime, strftime

In [2]:
if("notebooks" not in os.getcwd()):
    os.chdir("notebooks")
augurApp = augur.Application('../augur.config.json')
connect,list1,path = augurApp.github_issues()
token = list1[5]
DB_STR = 'mysql+pymysql://{}:{}@{}:{}/{}?charset=utf8'.format(
            list1[0], list1[1], list1[2],\
            list1[3], list1[4]
        )
db = s.create_engine(DB_STR)
Piper, path= augurApp.piper()

In [3]:
table_names = s.inspect(db).get_table_names()
print(table_names)
val = False
#mail_lists = True
git_repos = True
numb = 0
item = 1
columns2 = "augurlistID","backend_name","github_repo_url","project","last_run"
df_mail_list = pd.DataFrame(columns=columns2)
if("github_issues_2" not in table_names):
    #mail_lists = False
    git_repos = False
else:
    SQL = s.sql.text("""SELECT COUNT(*) FROM github_issues_2""")
    df7 = pd.read_sql(SQL, db)
    augurmsgID = int(df7.values)+1
    item = augurmsgID
if("github_issues_repo_jobs" in table_names):
    lists_createdSQL = s.sql.text("""SELECT project FROM github_issues_repo_jobs""")
    df1 = pd.read_sql(lists_createdSQL, db)
    print(df1)
    val = True
    val = db.engine.execute("""SELECT augurlistID FROM 
                                   github_issues_repo_jobs
                                   ORDER BY augurlistID DESC LIMIT 1""")
    for i in val:
        numb = i['augurlistID']
    
else:
    df_mail_list.to_sql(name="github_issues_repo_jobs",con=db,if_exists='replace',index=False,
                        dtype={'augurlistID': s.types.Integer(),
                               'backend_name': s.types.VARCHAR(length=300),
                               'github_repo_url': s.types.VARCHAR(length=300),
                               'project': s.types.VARCHAR(length=300),
                               'last_run': s.types.DateTime()
                        })
    lists_createdSQL = s.sql.text("""SELECT project FROM github_issues_repo_jobs""")
    df1 = pd.read_sql(lists_createdSQL, db)
    db.execute("ALTER TABLE github_issues_repo_jobs ADD PRIMARY KEY (augurlistID)")
    print("Created Table")
Base = declarative_base(db)
class table(Base):
    __tablename__ = 'github_issues_repo_jobs'
    __table_args__={'autoload':True}
    
Session = sessionmaker(bind=db)
session = Session()
res = session.query(table).all()

['github_issues', 'github_issues_sentiment_scores', 'github_pull_requests', 'github_pull_requests_sentiment_scores', 'issue_response_time', 'mail_lists', 'mail_lists_sentiment_scores', 'mailing_list_jobs']
Created Table


In [4]:
if "notebooks" in os.getcwd():
    os.chdir("..")
connect,list1,path = augurApp.github_issues()
print(path)
if(not os.path.exists(path)):
    file = open(path, "w+")
else:
    file = open(path, "r")
    print("yeah")
if (os.stat(path).st_size == 0):
    file.write("owner,repo_url\n")
    file.write("nodejs,\"CTC\"\n")
    #file.write("https://lists.opendaylight.org/pipermail/,\"archetypes-dev\"\n")
    print("Please enter the mailing lists and the links for them please")
    print("Going to the default mailing lists")

count = 0
for line in file:
    print(line)
    count+=1
    if(count == 2):
        break

runtime/git_repos.csv
yeah
owner,repo_url

torvalds,"linux"



In [5]:
print(df1['project'].values)

[]


In [6]:
# Url for the git repo to analyze
#repo_url = 'https://github.com/chaoss/grimoirelab-perceval'
#repo_url = 'CTC'
# Directory for letting Perceval clone the git repo
#repo_dir = 'CTC'
#own = 'nodejs'
# ElasticSearch instance (url)
#repo = 'grimoirelab-perceval'
# Create the 'commits' index in ElasticSearch
# Create a Git object, pointing to repo_url, using repo_dir for cloning

#32
#date = datetime.datetime(2017, 8, 7, 0, 0, 0,
#                                     tzinfo=dateutil.tz.tzutc())
'''
owner,repo_url
torvalds,"linux"
nodejs,"CTC"
nodejs,"TSC"
'''
'''
owner,repo_url
chaoss,"whitepaper"
chaoss,"prospector"
'''
k=0
df5 = pd.read_csv(path)
today = strftime("%a, %d %b %Y %H:%M:%S +0000", gmtime())
#for issues in repo.fetch(from_date=date):
columns1 = 'augurmsgID',"backend_name",'repo_link',"owner","repo","subject",\
          "status","category","issue_number","timestamp",\
          "issue_id","user","body"
df = pd.DataFrame(columns=columns1)
df.to_sql(name="github_issues_2", con=db,\
          if_exists='append',index=False,
           dtype={'augurmsgID': s.types.Integer,#0
                  'backend_name': s.types.VARCHAR(length=300),#1
                  'repo_link': s.types.VARCHAR(length=300),#2
                  'owner': s.types.VARCHAR(length=300),#3
                  'repo': s.types.VARCHAR(length=300),#4
                  'subject': s.types.VARCHAR(length=300),#5
                  'status': s.types.VARCHAR(length=10),#6
                  'category': s.types.VARCHAR(length=10),#7
                  'issue_number': s.types.Integer,#8
                  'date': s.types.DateTime(),#9
                  'issue_id': s.types.Integer,#10
                  'user': s.types.VARCHAR(length=100),#11
                  'body':s.types.TEXT#12              
           })

groups = df5.groupby('owner').groups
x = 0
new = False
for group in groups:
    own = group
    repo_url = (df5.loc[df5['owner'] == group]['repo_url']).tolist()
    print(own + '/' + repo_url[0])
    #break
    for y in repo_url:        
        repo = GitHub(owner=own,repository=y,api_token=token)
        inside = own + '/' + y
        if(inside not in df1['project'].values):
            new = True
            li = [[numb,"GitHub",'https://github.com/' + inside, inside,Piper.convert_date(today)]]
            df8 = pd.DataFrame(li,columns=columns2)
            df4 = df_mail_list.append(df8)
            df_mail_list = df4
            numb+=1
            froms = None
            #continue
        else:
            j = 0
            #print(today)
            time = Piper.convert_date(today)
            while(res[j].project!=inside):
                #print(res[j].project)
                j+=1
            now = res[j].last_run
            print(now)
            res[j].last_run = time
            session.commit()
            froms = now
            #froms = Piper.convert_date(now)
            #continue
            #"Thu, 5 Mar 2018 14:57:56 +0000"
        for issues in repo.fetch(from_date = froms):
            if 'pull_request' in issues['data']:
                #print(issues['data']['number'])
                continue
            else: 
                created = issues['data']['created_at']
                #print(type(froms))
                issue_time = created[:10] + " " + created[11:19]
                issue_time = dt.strptime(issue_time,"%Y-%m-%d %H:%M:%S")
                text = issues['data']['body']
                user = issues['data']['user']['login']
                num = issues['data']['number']
                id_num = issues['data']['id']
                store = [item,issues['backend_name'],issues['tag'],own,y,\
                         issues['data']['title'],issues['data']['state'],\
                         issues['category'],num,issue_time,\
                         id_num,user,text]
                df_user = pd.DataFrame([store],columns=columns1)
                if(froms == None):
                    df = df.append(df_user)
                    item+=1
                elif(issue_time > froms):
                    df = df.append(df_user)
                    item+=1
                    print("Yeah")
                for i in range(len(issues['data']['comments_data'])):
                    created = issues['data']['comments_data'][i]\
                    ['created_at']
                    issue_time = created[:10] + " " + created[11:19]
                    issue_time = dt.strptime(issue_time,"%Y-%m-%d %H:%M:%S")
                    temp=0
                    store[0] = item
                    store[8] = issues['data']['number']
                    store[9] = issue_time
                    store[10] = issues['data']['id']
                    store[11] = issues['data']['comments_data'][i]\
                    ['user_data']['login']
                    store[12] = issues['data']['comments_data'][i]['body']
                    df_user = pd.DataFrame([store],columns=columns1)
                    if(froms == None):
                        df = df.append(df_user)
                        item+=1
                    elif(issue_time > froms):
                        df = df.append(df_user)
                        item+=1

            if(df.shape[0] > 1000):
                df.to_sql(name='github_issues_2', con=db,\
                          if_exists='append',index=False)
                df = pd.DataFrame(columns=columns1)
                print("Broken")
                #break
            if(k%50 == 0):
                print(k)
            k+=1
            #if(k == 2):
            #    break
        if(df.shape[0] < 1000):
            df.to_sql(name='github_issues_2', con=db,\
                          if_exists='append',index=False)
            df = pd.DataFrame(columns=columns1)
            print("Broken")
        print("Total Number of issues: ", k)
        print(df.shape[0])
if(new == True):
    df_mail_list.to_sql(name='github_issues_repo_jobs',con=db,if_exists='append',index=False)

nodejs/CTC


2018-08-02 15:34:22 keanu-Inspiron-5567 root[18770] INFO Getting info for https://api.github.com/users/rvagg
2018-08-02 15:34:22 keanu-Inspiron-5567 root[18770] INFO Getting info for https://api.github.com/users/vkurchatkin
2018-08-02 15:34:23 keanu-Inspiron-5567 root[18770] INFO Getting info for https://api.github.com/users/benjamingr
2018-08-02 15:34:23 keanu-Inspiron-5567 root[18770] INFO Getting info for https://api.github.com/users/Trott
2018-08-02 15:34:23 keanu-Inspiron-5567 root[18770] INFO Getting info for https://api.github.com/users/joshgav
2018-08-02 15:34:24 keanu-Inspiron-5567 root[18770] INFO Getting info for https://api.github.com/users/Fishrock123


0


2018-08-02 15:34:25 keanu-Inspiron-5567 root[18770] INFO Getting info for https://api.github.com/users/jasnell
2018-08-02 15:34:26 keanu-Inspiron-5567 root[18770] INFO Getting info for https://api.github.com/users/addaleax
2018-08-02 15:34:26 keanu-Inspiron-5567 root[18770] INFO Getting info for https://api.github.com/users/ChALkeR
2018-08-02 15:34:27 keanu-Inspiron-5567 root[18770] INFO Getting info for https://api.github.com/users/bnoordhuis
2018-08-02 15:34:28 keanu-Inspiron-5567 root[18770] INFO Getting info for https://api.github.com/users/mhdawson
2018-08-02 15:34:28 keanu-Inspiron-5567 root[18770] INFO Getting info for https://api.github.com/users/williamkapke
2018-08-02 15:34:29 keanu-Inspiron-5567 root[18770] INFO Getting info for https://api.github.com/users/evanlucas
2018-08-02 15:34:29 keanu-Inspiron-5567 root[18770] INFO Getting info for https://api.github.com/users/MylesBorins
2018-08-02 15:34:30 keanu-Inspiron-5567 root[18770] INFO Getting info for https://api.github.com

2018-08-02 15:35:16 keanu-Inspiron-5567 root[18770] INFO Getting info for https://api.github.com/users/i5ting
2018-08-02 15:35:16 keanu-Inspiron-5567 root[18770] INFO Getting info for https://api.github.com/users/pesho
2018-08-02 15:35:17 keanu-Inspiron-5567 root[18770] INFO Getting info for https://api.github.com/users/thekemkid
2018-08-02 15:35:18 keanu-Inspiron-5567 root[18770] INFO Getting info for https://api.github.com/users/hxoht
2018-08-02 15:35:19 keanu-Inspiron-5567 root[18770] INFO Getting info for https://api.github.com/users/lamanabie
2018-08-02 15:35:20 keanu-Inspiron-5567 root[18770] INFO Getting info for https://api.github.com/users/CrabDude
2018-08-02 15:35:20 keanu-Inspiron-5567 root[18770] INFO Getting info for https://api.github.com/users/saschanaz
2018-08-02 15:35:21 keanu-Inspiron-5567 root[18770] INFO Getting info for https://api.github.com/users/leodutra
2018-08-02 15:35:21 keanu-Inspiron-5567 root[18770] INFO Getting info for https://api.github.com/users/DomVin

...

2018-08-02 15:50:20 keanu-Inspiron-5567 root[18770] INFO Getting info for https://api.github.com/users/akfish
2018-08-02 15:50:20 keanu-Inspiron-5567 root[18770] INFO Getting info for https://api.github.com/users/lkundrak
2018-08-02 15:50:21 keanu-Inspiron-5567 root[18770] INFO Getting info for https://api.github.com/users/kamilmaj94
2018-08-02 15:50:21 keanu-Inspiron-5567 root[18770] INFO Getting info for https://api.github.com/users/dabrace
2018-08-02 15:50:22 keanu-Inspiron-5567 root[18770] INFO Getting info for https://api.github.com/users/atonkyra
2018-08-02 15:50:22 keanu-Inspiron-5567 root[18770] INFO Getting info for https://api.github.com/users/platinumthinker
2018-08-02 15:50:23 keanu-Inspiron-5567 root[18770] INFO Getting info for https://api.github.com/users/chemelnucfin
2018-08-02 15:50:23 keanu-Inspiron-5567 root[18770] INFO Getting info for https://api.github.com/users/Xeru
2018-08-02 15:50:24 keanu-Inspiron-5567 root[18770] INFO Getting info for https://api.github.com/u

2018-08-02 15:50:59 keanu-Inspiron-5567 root[18770] INFO Getting info for https://api.github.com/users/XingZheZ
2018-08-02 15:51:00 keanu-Inspiron-5567 root[18770] INFO Getting info for https://api.github.com/users/Pr0Wolf29
2018-08-02 15:51:00 keanu-Inspiron-5567 root[18770] INFO Getting info for https://api.github.com/users/KISSMonX
2018-08-02 15:51:01 keanu-Inspiron-5567 root[18770] INFO Getting info for https://api.github.com/users/revmischa
2018-08-02 15:51:01 keanu-Inspiron-5567 root[18770] INFO Getting info for https://api.github.com/users/fntintash
2018-08-02 15:51:02 keanu-Inspiron-5567 root[18770] INFO Getting info for https://api.github.com/users/KAMiKAZOW
2018-08-02 15:51:02 keanu-Inspiron-5567 root[18770] INFO Getting info for https://api.github.com/users/ahmgeek
2018-08-02 15:51:03 keanu-Inspiron-5567 root[18770] INFO Getting info for https://api.github.com/users/kuehnelth
2018-08-02 15:51:03 keanu-Inspiron-5567 root[18770] INFO Getting info for https://api.github.com/use

2018-08-02 15:51:36 keanu-Inspiron-5567 root[18770] INFO Getting info for https://api.github.com/users/Robdeprop
2018-08-02 15:51:37 keanu-Inspiron-5567 root[18770] INFO Getting info for https://api.github.com/users/GijsWeterings
2018-08-02 15:51:38 keanu-Inspiron-5567 root[18770] INFO Getting info for https://api.github.com/users/jonnysandu
2018-08-02 15:51:38 keanu-Inspiron-5567 root[18770] INFO Getting info for https://api.github.com/users/dikiaap
2018-08-02 15:51:38 keanu-Inspiron-5567 root[18770] INFO Getting info for https://api.github.com/users/huntergregal
2018-08-02 15:51:39 keanu-Inspiron-5567 root[18770] INFO Getting info for https://api.github.com/users/AAndranik
2018-08-02 15:51:40 keanu-Inspiron-5567 root[18770] INFO Getting info for https://api.github.com/users/darksystem23
2018-08-02 15:51:40 keanu-Inspiron-5567 root[18770] INFO Getting info for https://api.github.com/users/akmittal
2018-08-02 15:51:41 keanu-Inspiron-5567 root[18770] INFO Getting info for https://api.gi

2018-08-02 15:52:12 keanu-Inspiron-5567 root[18770] INFO Getting info for https://api.github.com/users/xeronith
2018-08-02 15:52:13 keanu-Inspiron-5567 root[18770] INFO Getting info for https://api.github.com/users/parrotgeek1
2018-08-02 15:52:13 keanu-Inspiron-5567 root[18770] INFO Getting info for https://api.github.com/users/ddave09
2018-08-02 15:52:13 keanu-Inspiron-5567 root[18770] INFO Getting info for https://api.github.com/users/qioixiy
2018-08-02 15:52:14 keanu-Inspiron-5567 root[18770] INFO Getting info for https://api.github.com/users/luvkg
2018-08-02 15:52:15 keanu-Inspiron-5567 root[18770] INFO Getting info for https://api.github.com/users/AntiMoron
2018-08-02 15:52:15 keanu-Inspiron-5567 root[18770] INFO Getting info for https://api.github.com/users/robbyoconnor
2018-08-02 15:52:15 keanu-Inspiron-5567 root[18770] INFO Getting info for https://api.github.com/users/damquangtuan
2018-08-02 15:52:16 keanu-Inspiron-5567 root[18770] INFO Getting info for https://api.github.com/

RateLimitError: Rate limit exhausted.; 2290.0 seconds to rate reset